In [1]:
from helper import *
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri,numpy2ri
from rpy2.robjects.conversion import localconverter
pandas2ri.activate()
import pandas as pd
import os 
import pickle
import numpy as np

In [2]:
ro.r("""source('helper.R')""")

R[write to console]: Loading required package: Matrix

R[write to console]: Loading required package: sp

R[write to console]: This is INLA_24.04.01 built 2024-04-01 19:43:22 UTC.
 - See www.r-inla.org/contact-us for how to get help.
 - List available models/likelihoods/etc with inla.list.models()
 - Use inla.doc(<NAME>) to access documentation

R[write to console]: 
Attaching package: ‘zoo’


R[write to console]: The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: This is rSPDE 2.3.3.9000
- See https://davidbolin.github.io/rSPDE for vignettes and manuals.



In [3]:


max_id=get_max("Log")
logger_path=os.path.join("Log",f"{max_id}.plot")

exp_id=20
path=f"/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/fits"
results = []
for file in os.listdir(path):
     fit_path=os.path.join(path,file)
     ro.r(f"""

          xx<-readRDS("{fit_path}")
          result<-parse_result(xx$fit,xx$family)
          """)
     df=ro.r("result$df")
     likelihood=ro.r("unlist(result$like)")
     family={"model":ro.r("xx$family$model")[0],
             "Y":ro.r("xx$family$Y")[0]}
     startValue={"prior.std.dev.nominal":ro.r("xx$start_value$prior.std.dev.nominal")[0],
                    "prior.range.nominal":ro.r("xx$start_value$prior.range.nominal")[0],
                    "rspde.order":ro.r("xx$start_value$rspde.order")[0]}
     
     results.append({"df":ro.conversion.rpy2py(df),
                     "likelihood":likelihood,
                     "family":family,
                     "startValue":startValue,
                     "id":file.split("_")[1][:-4]})

In [4]:
models=dict()
for result in results:
    key=f"{result['family']['model']}_{result['family']['Y']}"
    if key not in models:
        models[key]=list()
    models[key].append(result)

save the parsed result

In [5]:

pickle.dump(models,open(f"/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/models.pkl","wb"))

In [7]:
models.keys()

dict_keys(['rough_stochvol.t', 'rough_stochvol', 'rough_stochvol.nig'])

In [12]:
start_values_likelihood={}
model_key="rough_stochvol"
models[model_key]=sorted(models[model_key],key=lambda x: f"""({x["startValue"]["prior.std.dev.nominal"]:.0f},{x["startValue"]["prior.range.nominal"]:.0f},{x["startValue"]["rspde.order"]:.0f})""")
for fit in models[model_key]:
    
    statValue=fit["startValue"]
    key=f"""({statValue["prior.std.dev.nominal"]:.0f},{statValue["prior.range.nominal"]:.0f},{statValue["rspde.order"]:.0f})"""
    start_values_likelihood[key]=fit["likelihood"]
    
df=pd.DataFrame(start_values_likelihood)
df.index=df.index+1
df=df.T
df = df.style.format(decimal='.', precision=2)
print(df.to_latex())

\begin{tabular}{lr}
 & 1 \\
(0,212,5) & 6609.55 \\
(0,213,4) & 6608.08 \\
(0,215,2) & 6608.81 \\
(0,231,2) & 6607.89 \\
(0,88,2) & 6609.42 \\
(1,146,5) & 6624.75 \\
(1,147,2) & 6627.20 \\
(1,148,6) & 6625.48 \\
(1,149,4) & 6610.51 \\
(1,153,4) & 6610.25 \\
(1,162,6) & 6615.39 \\
(1,177,3) & 6626.83 \\
(1,179,2) & 6626.80 \\
(1,180,2) & 6626.97 \\
(1,199,3) & 6609.38 \\
(1,199,4) & 6627.84 \\
(1,203,3) & 6626.77 \\
(1,220,3) & 6609.39 \\
(1,262,4) & 6611.17 \\
(1,291,2) & 6626.37 \\
(1,295,3) & 6608.62 \\
(1,296,6) & 6626.30 \\
(1,31,5) & 6612.54 \\
(1,62,4) & 6613.03 \\
(1,73,5) & 6627.66 \\
(1,77,5) & 6614.87 \\
(2,184,2) & 6626.03 \\
\end{tabular}



In [88]:
models.keys()

dict_keys(['rs_stochvol', 'rough_stochvolln', 'rough_stochvol.t', 'rough_stochvol', 'rough_stochvol.nig'])

In [89]:
model_key="rs_stochvol"
max_id_=np.argmax([max(x["likelihood"]) for x in models[model_key] ])
print("config id",models[model_key][max_id_]["id"])
print("run id",np.argmax(models[model_key][max_id_]["likelihood"]))

config id 150
run id 0


In [84]:
np.argsort([max(x["likelihood"]) for x in models[model_key] ])

array([10, 23,  0, 19,  3, 11, 16, 25, 29,  1, 28, 24, 14, 26, 15, 12, 21,
       17, 22, 27,  2,  5,  4, 18, 20, 13,  9,  6,  8,  7])

In [85]:
models[model_key][8]["id"]

'65'

## gather all models result into one Table

In [104]:
records=[]
for model_key in ["rough_stochvol","rough_stochvol.t","rough_stochvol.nig","rough_stochvolln"]:
    if model_key=="rough_stochvol.t":
        max_id_=np.argsort([max(x["likelihood"]) for x in models[model_key] ])[-2]
    else:
        max_id_=np.argmax([max(x["likelihood"]) for x in models[model_key] ])
    fits_for_a_start_values=models[model_key][max_id_]
    

    idd=np.argmax(fits_for_a_start_values["likelihood"])
    df=models[model_key][max_id_]["df"]
    tmp_df=df.loc[[x for x in df.index if x.startswith(f"e{idd+1}")]]
    new_key=model_key.replace("_","-")
    indexx=[f"{new_key}_{x[2:]}" for x in tmp_df.index]
    tmp_df.index=indexx
    records.append(tmp_df)

    

In [105]:
np.argsort(fits_for_a_start_values["likelihood"])

array([0])

In [108]:

df = pd.concat(records).style.format(decimal='.', precision=2)
print(df.to_latex())

\begin{tabular}{lrrrrrr}
 & mean & sd & 0.025quant & 0.5quant & 0.975quant & mode \\
rough-stochvol_std.dev & 0.56 & 0.06 & 0.46 & 0.56 & 0.69 & 0.55 \\
rough-stochvol_range & 25.46 & 16.34 & 7.74 & 21.03 & 67.68 & 16.40 \\
rough-stochvol_nu & 0.09 & 0.02 & 0.04 & 0.09 & 0.14 & 0.09 \\
rough-stochvol.t_std.dev & 0.74 & 0.24 & 0.44 & 0.68 & 1.38 & 0.62 \\
rough-stochvol.t_range & 23.78 & 12.47 & 6.25 & 21.53 & 54.84 & 18.09 \\
rough-stochvol.t_nu & 0.22 & 0.18 & 0.03 & 0.16 & 0.70 & 0.08 \\
rough-stochvol.nig_std.dev & 0.69 & 0.10 & 0.54 & 0.67 & 0.93 & 0.65 \\
rough-stochvol.nig_range & 28.69 & 8.26 & 15.25 & 27.78 & 46.99 & 24.94 \\
rough-stochvol.nig_nu & 0.24 & 0.09 & 0.09 & 0.23 & 0.45 & 0.21 \\
rough-stochvolln_std.dev & 1.17 & 0.06 & 1.04 & 1.18 & 1.29 & 1.19 \\
rough-stochvolln_range & 9.79 & 1.07 & 7.74 & 9.81 & 11.87 & 10.16 \\
rough-stochvolln_nu & 0.48 & 0.03 & 0.42 & 0.48 & 0.53 & 0.48 \\
\end{tabular}



#### inspect the stability

In [15]:
models["rough_stochvol"][1]

{'df':               mean        sd  0.025quant  0.5quant  0.975quant     mode
 e1std.dev  1.57530  0.075387     1.43159   1.57361     1.72608  1.57474
 e1range    6.61859  0.334054     5.95027   6.62636     7.25480  6.65180
 e1nu       1.59052  0.030999     1.53560   1.58826     1.65564  1.57842,
 'likelihood': array([6608.076]),
 'family': {'model': 'rough', 'Y': 'stochvol'},
 'startValue': {'prior.std.dev.nominal': 0.3369386751908445,
  'prior.range.nominal': 212.7656998098828,
  'rspde.order': 4},
 'id': '7'}

In [46]:
models.keys()

dict_keys(['rs_stochvol', 'rough_stochvolln', 'rough_stochvol.t', 'rough_stochvol', 'rough_stochvol.nig'])

In [67]:
model_key="rough_stochvolln"

In [68]:


all=[(x["startValue"],x["df"]["mean"],x["likelihood"]) for x in models[model_key]]
df=pd.concat([x[1] for x in all],axis=1)

df.columns=[f"""({x[0]["rspde.order"]}/{x[0]["prior.range.nominal"]:.4f}/{x[0]["prior.std.dev.nominal"]:.4f})""" for x in all]
df=df.T
df.columns=[x[2:] for x in df.columns]
df["likelihood"]=np.array([x[2] for x in all])

df=df.sort_values("likelihood",ascending=False)
print(df.to_latex(float_format="%.3f"))

\begin{tabular}{lrrrr}
\toprule
 & std.dev & range & nu & likelihood \\
\midrule
(6/31.1054/0.4286) & 1.172 & 9.791 & 0.476 & 6601.487 \\
(4/61.0968/0.6265) & 1.438 & 6.748 & 1.378 & 6598.949 \\
(2/134.2289/0.2284) & 1.190 & 11.939 & 0.653 & 6596.105 \\
(2/285.0453/0.4559) & 1.309 & 8.525 & 1.466 & 6590.954 \\
(3/31.9357/0.6857) & 1.448 & 9.994 & 1.166 & 6590.834 \\
(2/12.0289/0.5494) & 1.692 & 11.149 & 1.152 & 6587.893 \\
(3/65.6989/1.6841) & 1.731 & 7.964 & 0.927 & 6587.862 \\
(5/276.6435/0.1497) & 1.254 & 3.216 & 2.000 & 6563.986 \\
(5/134.8579/1.4616) & 1.253 & 52.966 & 1.015 & 6547.063 \\
(2/203.1846/0.3531) & 1.156 & 35.657 & 1.991 & 6545.283 \\
(5/68.9412/0.4789) & 1.269 & 56.640 & 1.246 & 6535.706 \\
(3/123.5039/0.7484) & 1.200 & 63.546 & 1.182 & 6530.076 \\
(6/138.1205/0.5539) & 1.482 & 80.168 & 1.299 & 6520.457 \\
(6/121.2197/0.4731) & 1.286 & 88.706 & 0.833 & 6520.381 \\
(6/152.0343/0.5363) & 1.422 & 95.664 & 1.153 & 6512.806 \\
(2/179.8547/0.5478) & 1.773 & 114.373 & 1.056 